In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

Read in the data

In [ ]:
FFP = True
ffpname = ''
if FFP:
    ffpname='_kp'

batch_size = 128
num_classes = 2

# OK, now we need to load in the training data from both snapshots

train_x_b1_1 = np.load('b1p.01XY_train'+ffpname+'_ss1_sh_64_ims_128.npy')
train_x_b1_2 = np.load(path + 'b1p.01XY_train'+ffpname+'_ss1_sh_64_ims_128_2.npy')
train_x_b1= np.concatenate((train_x_b1_1, train_x_b1_2), axis=2)

train_x_b01_1 = np.load('b.1p.01XY_train'+ffpname+'_ss1_sh_64_ims_128.npy')
train_x_b01_2 = np.load('b.1p.01XY_train'+ffpname+'_ss1_sh_64_ims_128_2.npy')
train_x_b01= np.concatenate((train_x_b01_1, train_x_b01_2), axis=2)

trshape = train_x_b1.shape

# input image dimensions
img_rows, img_cols = trshape[0], trshape[1]

x_train = np.zeros([trshape[2]*2, img_rows, img_rows])
x_train[::2, :, :] = np.transpose(train_x_b01)
x_train[1::2, :, :] = np.transpose(train_x_b1)

y_train = np.zeros([trshape[2]*2])
y_train[::2] = np.zeros([trshape[2]])
y_train[1::2] = np.ones([trshape[2]])

#load in the test data from both snapshots

test_x_b1_1 = np.load('b1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128.npy')
test_x_b1_2 = np.load('b1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128_2.npy')
test_x_b1 = np.concatenate((test_x_b1_1, test_x_b1_2), axis=2)

test_x_b01_1 = np.load('b.1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128.npy')
test_x_b01_2 = np.load('b.1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128_2.npy')
test_x_b01 = np.concatenate((test_x_b01_1, test_x_b01_2), axis=2)

teshape = test_x_b1.shape

x_test = np.zeros([teshape[2]*2, img_rows, img_rows])
x_test[::2, :, :] = np.transpose(test_x_b01)
x_test[1::2, :, :] = np.transpose(test_x_b1)

y_test = np.zeros([teshape[2]*2])
y_test[::2] = np.zeros([teshape[2]])
y_test[1::2] = np.ones([teshape[2]])


Reformatting data for keras

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Build the network

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

A summary of the network structure

In [ ]:
model.summary()

Train the network

In [ ]:
modelname = 'PB19_network.dms'
if FFP:
    modelname = 'PB19_FFP_network.dms'

epochs = 12
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

model.save(modelname)